# **Decision Tree Implementation**

**Escuela de Ingeniería en Computación**

**IC6200 - Inteligencia Artificial - Tarea 4 - Árboles de Decisión**

**Estudiantes:**

Gerald Núñez Chavarría - 2021023226

Sebastián Arroniz Rojas - 2021108521

Sebastián Bermúdez Acuña - 2021110666

**Profesor:**

Kenneth Obando Rodríguez

**Fecha de entrega:**

01/04/2024

## **1. Clase Nodo**

Lo primero es crear una clase nodo que tenga los atributos necesarios para un árbol de decisión: feature, umbral, gini, sample_quantity, value, left and right. 